In [1]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#reading csv file
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#!pip install xgboost

In [4]:
#splitting data into x
x=df.iloc[:,3:13].values


In [5]:
#splitting data into y
y=df.iloc[:,13]


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [7]:
labelencoder_cs=LabelEncoder()

In [8]:
x[:,1]=labelencoder_cs.fit_transform(x[:,1])

In [9]:
labelencoder_gender=LabelEncoder()

In [10]:
x[:,2]=labelencoder_gender.fit_transform(x[:,2])

In [11]:
#onehotencoder=OneHotEncoder(categorical_features=[1])

In [12]:
#onehotencoder.fit_transform(x).toarray()

In [13]:
import xgboost

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=100)

In [16]:
from xgboost import XGBClassifier

In [17]:
xgbc=XGBClassifier()

In [18]:
xgbc.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [19]:
pred=xgbc.predict(x_test)

In [20]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [22]:
cm=confusion_matrix(y_test,pred)
cm

array([[2316,   60],
       [ 349,  275]])

In [34]:
score=accuracy_score(y_test,pred)*100
score

86.36666666666667

In [24]:
from sklearn.model_selection import cross_val_score

In [35]:
cross_val_score(
    xgbc,
    x,
    y,
    groups=None,
    scoring=None,
    cv=10,
    n_jobs=None,
    verbose=0,
    fit_params=None,
    pre_dispatch='2*n_jobs',
    error_score='raise-deprecating',
).mean()

0.8638996200996202

In [36]:
#min range: 85% while max range : 87.88% and mean : 86.37%

In [37]:
from sklearn.model_selection import GridSearchCV

In [51]:
paramters=[{'C': [1,10,100,1000],'kernel':['linear']},
          {'C': [1,10,100,1000],'kernel':['rbf'],'gamma':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}]
gridcv=GridSearchCV(estimator=xgbc,
    param_grid=paramters,
    scoring='accuracy',
    n_jobs=None,
    iid='warn',
    refit=True,
    cv=10,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score='raise-deprecating',
    return_train_score=False)


In [52]:
gridcv.fit(x_train,y_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000],
                          'gamma': [0.1, 

In [57]:
#best parameter in gridcv 
accuracy1=gridcv.best_params_
accuracy1

{'C': 1, 'kernel': 'linear'}

In [59]:
#from sklearn.metrics import accuracy_score
#accuracy=accuracy_score(y_test,pred)
accuracy=gridcv.best_score_
accuracy*100

86.55714285714285